In [ ]:
custom_prompt_template_1 = """
You are a helpful and accurate assistant, but you are also a meticulous researcher. Your task is to provide a final, well-informed answer to the user's question. To ensure your answer is accurate and complete, you must perform a thorough search and gather information from a few different sources.

You have access to the following tools:

{tools}

You must respond using a specific format.

**If you need to use a tool to find an answer:**
Thought: I need to use a tool to conduct thorough research. I will perform a search for [specific topic of research].
Action: [the name of the tool to use, must be one of {tool_names}]
Action Input: [The input for the tool. Use a single line.]

**Once you have finished your research and gathered sufficient information to provide a complete and accurate answer, you can stop using tools. Instead, respond in this exact format:**
Thought: I have completed my research and am confident I can provide a complete and accurate final answer.
Final Answer: [The final, well-informed answer to the user's question, synthesized from all the information you have gathered.]

Begin!

Question: {input}
{agent_scratchpad}
"""

In [ ]:

custom_prompt_template_2= """
You are a helpful and accurate assistant. Your task is to respond to a user's question.

You have access to the following tools:

{tools}

You must respond using a specific format.

**If you need to use a tool to find an answer:**
Thought: Do I need to use a tool? Yes. I should use the [tool_name] tool to find the answer.
Action: [the name of the tool to use, must be one of {tool_names}]
Action Input: [The input for the tool. Use a single line.]

**If you have the final answer, do not use any tools. Instead, respond in this exact format:**
Thought: I have a final answer.
Final Answer: [The final, concise answer to the user's question]

Begin!

Question: {input}
{agent_scratchpad}
"""

In [ ]:
import os
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.chat_models import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# 1. Define the tools the agent can use
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

# 2. Initialize the Ollama model
# Ensure 'llama2:latest' is pulled via `ollama pull llama2:latest`.
# This model is more capable than smaller ones but still benefits from careful prompting.
llm = ChatOllama(
    model="llama2:latest",
    temperature=0
)

# 3. Create a custom prompt to explicitly define the format.
# This is the most important fix for parsing errors with local models.
# It guides the model to produce output in the exact format the parser expects.
# ... (your existing code for tool and llm setup)

# Create a custom prompt to explicitly define the format.
# This is the most important fix for parsing errors with local models.


custom_prompt_template = custom_prompt_template_1



# Create a PromptTemplate instance from the custom template
custom_prompt = PromptTemplate.from_template(custom_prompt_template)

# ... (rest of your code for creating the agent and executor)

# Create a PromptTemplate instance from the custom template
custom_prompt = PromptTemplate.from_template(custom_prompt_template)

# 4. Create the agent
# Pass the custom prompt to the agent builder.
agent = create_react_agent(llm, tools, custom_prompt)

# 5. Create an agent executor to run the agent.
# The `handle_parsing_errors=True` parameter is crucial. 
# It tells the executor to pass parsing errors back to the LLM, giving it another chance to fix its output.
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 6. Example interaction
user_query = "Explain calculs in brief?"
print(f"User Query: {user_query}\n")

try:
    response = agent_executor.invoke({"input": user_query})
    print(f"\nFinal Response: {response['output']}")
except Exception as e:
    print(f"\nAn error occurred: {e}")

User Query: Explain calculs in brief?



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? Yes. I should use the duckduckgo_search tool to find the answer.
Action Input: Calculus is a branch of mathematics that deals with the study of continuous change, including the concepts of limits, derivatives, and integrals.

Thought: I have a final answer.
Final Answer: Calculus is a fundamental concept in mathematics that helps us understand how things change over time or distance. It involves the study of rates of change, slopes of curves, and accumulation of quantities.

> Finished chain.

Final Response: Calculus is a fundamental concept in mathematics that helps us understand how things change over time or distance. It involves the study of rates of change, slopes of curves, and accumulation of quantities.
